In [1]:
from edgar import set_identity

In [2]:
# must be set to comply with SEC regulations

set_identity("Neal Lockhart nhl3388@uncw.edu")

[16:55:48] INFO     Identity of the Edgar REST client set to [Neal Lockhart nhl3388@uncw.edu]           ]8;id=311121;file://c:\Users\Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\edgar\core.py\core.py]8;;\:]8;id=587157;file://c:\Users\Dave\AppData\Local\Programs\Python\Python311\Lib\site-packages\edgar\core.py#161\161]8;;\

## Filings: 10-k

### Markdown

In [3]:
from edgar import *

In [45]:
filings = Company("NOW").get_filings(form="10-K").latest(2)

In [5]:
filings[0]

╭─────────────────────────────────────── Snowflake Inc. [1640147] 10-K 📊 ────────────────────────────────────────╮
│ ╭──────────────────────┬────────────╮                                                                           │
│ │ 0001640147-24-000101 │ 2024-03-26 │                                                                           │
│ ╰──────────────────────┴────────────╯                                                                           │
│ ╭────────────────────────────────────────────────────────────────────────────────────────────╮                  │
│ │ Links: 🏠 Homepage 📄 Primary Document 📜 Full Submission Text                             │                  │
│ ├────────────────────────────────────────────────────────────────────────────────────────────┤                  │
│ │ 🏠 https://sec.gov/Archives/edgar/data/1640147/0001640147-24-000101-index.html             │                  │
│ │ 📄 https://sec.gov/Archives/edgar/data/1640147/000164014724000101/snow-202

In [46]:
md = filings[0].sections()

In [49]:
md

['now-20231231',
 'UNITED STATES',
 'SECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549',
 'FORM 10-K',
 '☒        Annual Report pursuant to Section 13 or 15(d) of the Securities Exchange Act of 1934',
 'For the fiscal year ended December 31, 2023',
 'OR',
 '☐        Transition report pursuant to Section 13 or 15(d) of the Securities Exchange Act of 1934',
 'Commission File Number: 001-35580',
 'SERVICENOW, INC.\n(Exact name of registrant as specified in its charter)',
 'Delaware                               20-2056195                 \n(State or other jurisdiction of        (I.R.S. Employer           \nincorporation or organization)         Identification Number)',
 'ServiceNow, Inc.',
 '2225 Lawson Lane',
 'Santa Clara, California 95054\n(408) 501-8550\n(Address, including zip code, and telephone number, including area code, of registrant’s principal executive offices)\nSecurities registered pursuant to Section 12(b) of the Act:',
 'Title of each class                       

In [47]:
#search for the word "risk" in the 10-k sections list
a_list=[]
for i in range(len(md)):
    if "item 1a." in md[i].lower():
        index_start=i
    if "item 1b." in md[i].lower():
        index_end=i

md[index_start:index_end]



['ITEM 1A.       RISK FACTORS',
 'Investing in our securities involves risks. You should carefully consider the risks and uncertainties described below, together with the other information in this Annual Report on Form 10-K, before making an investment decision. The occurrence of any of the following risks, or additional risks and uncertainties not presently known to us or that we currently believe to be immaterial, could materially and adversely affect our business, financial condition or results of operations or cause our stock price to decline. The following risks have been grouped by categories and are not in order of significance or probability of occurrence.',
 'Risk Factors Summary\nThis summary provides an overview of the risks we face and should not be considered a substitute for the more fulsome risk factors discussed immediately following this summary. \n•Risks Related to Our Ability to Grow Our Business\n•Laws, regulations and customer expectations regarding the use, storag

In [48]:
index_end

63

In [42]:
index_end

90

In [6]:
with open('markdown.md', 'w', encoding='utf-8') as f:
    chunk = 8192
    for i in range(0, len(md), chunk):
        f.write(md[i : i+chunk])

### HTML

In [7]:
html = filings[0].html()

In [8]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, "html.parser")

In [11]:
soup

<?xml version='1.0' encoding='ASCII'?>
<!--XBRL Document Created with the Workiva Platform-->
<!--Copyright 2024 Workiva-->
<!--r:5a77c039-7ace-40cd-8ebe-2c5c952cefcc,g:390ba4e4-144a-4d29-9fe6-e7b0d7caff3c,d:7f547f2e137c4b64b6aad752df863ff6-->
<html xml:lang="en-US" xmlns="http://www.w3.org/1999/xhtml" xmlns:country="http://xbrl.sec.gov/country/2023" xmlns:dei="http://xbrl.sec.gov/dei/2023" xmlns:ecd="http://xbrl.sec.gov/ecd/2023" xmlns:iso4217="http://www.xbrl.org/2003/iso4217" xmlns:ix="http://www.xbrl.org/2013/inlineXBRL" xmlns:ixt="http://www.xbrl.org/inlineXBRL/transformation/2020-02-12" xmlns:ixt-sec="http://www.sec.gov/inlineXBRL/transformation/2015-08-31" xmlns:link="http://www.xbrl.org/2003/linkbase" xmlns:snow="http://www.snowflake.com/20240131" xmlns:srt="http://fasb.org/srt/2023" xmlns:us-gaap="http://fasb.org/us-gaap/2023" xmlns:xbrldi="http://xbrl.org/2006/xbrldi" xmlns:xbrli="http://www.xbrl.org/2003/instance" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://

In [13]:
with open('html.txt', 'w', encoding='utf-8') as f:
    f.write(soup.get_text())

<_io.TextIOWrapper name='html.txt' mode='w' encoding='utf-8'>

In [14]:
risk_factors_header = soup.find(string="ITEM 1A. RISK FACTORS")

In [15]:
element = risk_factors_header.find_parent("div")

In [16]:
next_element = element.find_next_sibling()

In [17]:
element.get_text()

'ITEM 1A. RISK FACTORS'

In [18]:
next_element.get_text()

'Our operations and financial results are subject to various risks and uncertainties, including those described below. You should consider and read carefully all of the risks and uncertainties described below, together with all of the other information contained in this Annual Report on Form 10-K, including the sections titled “Special Note about Forward-Looking Statements” and “Management’s Discussion and Analysis of Financial Condition and Results of Operations” and our consolidated financial statements and related notes, before making an investment decision. The risks described below are not the only ones we face. The occurrence of any of the following risks or additional risks and uncertainties not presently known to us or that we currently believe to be immaterial could materially and adversely affect our business, financial condition, results of operations, or growth prospects. In such case, the trading price of our common stock could decline. You should not interpret our disclos

In [116]:
next_element.find_next_sibling().get_text()

'Risks Related to Our Business and Operations'

In [107]:
def get_full_section_text(start_header):
    section_text = []
    current_element = start_header.find_parent("div")
    
    while current_element:
        next_element = current_element.find_next_sibling()
        
        # Check if we've hit the next major section
        if next_element and any(text.strip().startswith("ITEM ") 
                              for text in next_element.stripped_strings):
            break
            
        if next_element:
            section_text.append(next_element.get_text(strip=True))
            current_element = next_element
        else:
            break
    
    return ' '.join(section_text)

risk_factors_header = soup.find(string="ITEM 1A. RISK FACTORS")
full_risk_section = get_full_section_text(risk_factors_header)

In [109]:
with open('text.txt', 'w') as f:
    f.write(full_risk_section)